In [36]:
from gensim.models import word2vec
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

corpus = [
          'Text of the first document.',
          'Text of the second document made longer.',
          'Number three.',
          'This is number four.',
]
# we need to pass splitted sentences to the model
tokenized_sentences = [sentence.split() for sentence in corpus]
model = word2vec.Word2Vec(tokenized_sentences, min_count=1)

In [26]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [23]:
words, word_to_vec_map = read_glove_vecs('./glove.6B.50d.txt')

In [27]:
happy_train = pd.read_csv('./CrawlComment/final_happy.csv', sep='\t', header=None)
angry_train = pd.read_csv('./CrawlComment/final_angry.csv', sep='\t', header=None)
sad_train = pd.read_csv('./CrawlComment/final_sad.csv', sep='\t', header=None)
prank_train = pd.read_csv('./CrawlComment/final_prank.csv', sep='\t', header=None)
frames = [happy_train, angry_train, sad_train, prank_train]

df = pd.concat(frames, ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12864 entries, 0 to 12863
Data columns (total 2 columns):
0    12864 non-null object
1    12864 non-null int64
dtypes: int64(1), object(1)
memory usage: 201.1+ KB


In [33]:
X = df[0]
y = df[1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, shuffle=True)

print(X_train.shape)
print(y_train.shape)
print(y_train.unique())
print(y_train.value_counts())

(9004,)
(9004,)
[1 2 3 0]
3    2718
0    2397
1    2140
2    1749
Name: 1, dtype: int64


In [70]:
def avg_vec(sentence):
    words = sentence.lower().split()
    avg = np.zeros((50,0))
    total = 0
    
    for word in words:
        try:
            vec = word_to_vec_map[word]
        except:
            print(word)
            vec = word_to_vec_map['unk']
            
        total += vec
        
    avg = total/len(words)
    return avg

def check_word(sentence):
    words = sentence.lower().split()
    
    for word in words:
        try:
            vec = word_to_vec_map[word]
        except:
            print(word)

In [75]:
check_word('hello my friend cant')

In [67]:
# m = y_train.shape[0]
# avg_train = np.zeros((y_train.shape[0], 0))

# for i in range(m):
#     avg_train[i] = avg_vec(X_train[i])

# avg_train

# SGD_clf = SGDClassifier(loss='hinge', penalty='l2',
#                           alpha=1e-4, random_state=42,
#                           max_iter=100, tol=None)
# SGD_clf.fit()

ValueError: could not broadcast input array from shape (50) into shape (0)